In [1]:
import pdfplumber
import pandas as pd
import re
import xlwings as xw
import pyperclip
import keyboard
import time

In [2]:
def get_pdf_page(page_num):
    text = ''
    with pdfplumber.open(path) as pdf:      
        first_page = pdf.pages[page_num-1]
        #有方框的才能提取表格
#         if len(first_page.extract_tables())!=0:
#             print(first_page.extract_tables())
        temp_list = first_page.extract_words()
        for dic in temp_list:
            dic['text'] = re.sub('\(cid:\d+\)','',dic['text'])
    return temp_list

In [3]:
def find_in_page(list1,page):
    for ele in list1:
        for element in page:
            if ele == element['text']:
                print(element['text']+'  左边'+str(element['x0'])+'  上面'+str(element['top'])+'  右边'+str(element['x1']))

In [4]:
def spacing(a,b):
    return a-b if a>b else b-a

In [5]:
# sheet列名转换
def get_column_right(column,add_num):
    # 将列标转换为10进制数
    column_number = 0
    for i in range(len(column)):
        column_number = column_number * 26 + (ord(column[i]) - ord('A') + 1)
    
    # 对转换后的10进制数加步长
    column_number += add_num
    
    # 将得到的结果转换为Excel的列标
    result = ''
    while column_number > 0:
        remainder = (column_number - 1) % 26
        result = chr(ord('A') + remainder) + result
        column_number = (column_number - 1) // 26
    
    return result

In [6]:
app = xw.App(visible=True, add_book=False)

path = 'D:/Backup/Downloads/yh.pdf'
wb = app.books.open('E:/桌面/银行名单.xlsx')
# 连接到已经打开的 Excel 文件
# wb = xw.Book('example.xlsx')
sheet = wb.sheets[0]

# 通用模板

In [49]:
# 换pdf修改
path = 'D:/Backup/Downloads/yh.pdf'

sheet = wb.sheets[5] # 5
# 换页修改
tlist = get_pdf_page(172)
start_line = 108
start_zimu = 'C'# 填数字的开始列名
need_ele = ['140','276']
find_in_page(need_ele,tlist)

In [50]:
# 可变参数
first_range = [255,280] # 第一列colomn index范围
num_start_range = first_range[1] # 后边的跟班数字起始位置，默认为index右坐标
start_top = 195 # 起始top
max_top = 800 #最底部坐标

max_x0 = 550 #最右边坐标
merge_lag = 5 #同行合并误差
diff_lag = 120 #不同行的间距，用于补充空行
lag_x = 10 #用于补充空值
duiqi = 'right' #数字是右对齐

temp_list = []
for element in tlist:
    if element['x0']>first_range[0] and element['x0']<first_range[1] and element['top']>start_top and element['top']<max_top:
        temp_list.append({'text':element['text'],'x0':element['x0'],'top':element['top']})

# 先按top排序
temp_list.sort(key=lambda x: x.get('top'),reverse=False)
# 再按x0合并
first_coloum =[]
merge_list = []
for i in range(0,len(temp_list)):
    if i != len(temp_list)-1 and spacing(temp_list[i]['top'],temp_list[i+1]['top'])<merge_lag:
        merge_list.append(temp_list[i])
    else:
        merge_list.append(temp_list[i])
        merge_list.sort(key=lambda x: x.get('x0'))
        merge_text = ''.join([d.get('text') for d in merge_list])
        first_coloum.append({'text': merge_text,'x0':merge_list[0]['x0'],'top':merge_list[0]['top']})
        merge_list = []
# 补充空行
for i in range(0,100):
    if i >= len(first_coloum)-1:
        break
    if first_coloum[i+1]['top'] - first_coloum[i]['top']>diff_lag:
        first_coloum.insert(i+1,{'text':'','x0':first_coloum[i]['x0'],'top':first_coloum[i]['top']+diff_lag})
for i in range(0,100):
    if first_coloum[-1]['top']+diff_lag>max_top:
        break
    first_coloum.append({'text':'','x0':first_coloum[-1]['x0'],'top':first_coloum[-1]['top']+diff_lag})
# 去除非中文和数字
for ele in first_coloum:
    ele['text'] = re.sub(r'[^\u4e00-\u9fa5\d\(\)\.、]+', '', ele['text'])
# 按first_coloum拿之后的数据
for ele in first_coloum:
    ele['num'] = []
    for element in tlist:
        if element['x0']>num_start_range and spacing(element['top'],ele['top'])<merge_lag and element['text']!='':
            ele['num'].append(element)
    ele['num'].sort(key=lambda x: x.get('x0'))

# 补充空值
max_column = 0
duiqi = 'x1' if duiqi=='right' else 'x0'
for ele in first_coloum:
    if len(ele['num'])>max_column:
        max_column=len(ele['num'])
for i in range(0,max_column):# i = 0 - 4
    list_temp = []
    for j in range(0,len(first_coloum)):
        if len(first_coloum[j]['num'])>i:
            list_temp.append(first_coloum[j]['num'][i])
    min_num = min([ele[duiqi] for ele in list_temp])
    for j in range(0,len(first_coloum)):
        if len(first_coloum[j]['num'])<=i:
            first_coloum[j]['num'].insert(i,{'text':''})
        else:
            if first_coloum[j]['num'][i][duiqi] > min_num + lag_x:
                first_coloum[j]['num'].insert(i,{'text':''})
    
i = start_line
# 填数字
for ele in first_coloum:
    sheet.range(f'{start_zimu}{i}').value = ele['text']
    j = 1
    for element in ele['num']:
        zimu = get_column_right(start_zimu,j)
        sheet.range(f'{zimu}{i}').value = element['text']
        j += 1
    i+=1
    
print('ok')

ok


# 手动工具系列

In [75]:
# 临时工具去空
while True:
    if keyboard.is_pressed('shift'):
        app.selection.delete(shift='left')
        time.sleep(0.3)
    if keyboard.is_pressed('caps lock'):
        app.selection.delete(shift='up')
        time.sleep(0.3)
    if keyboard.is_pressed('win'):
        break

In [151]:
# 寻找遗失的行
miss_line =[]
miss_line_new = [340,375]
tlist.sort(key=lambda x: x.get('top'),reverse=False)
for element in tlist:
    if element['x0']>miss_line_new[0] and element['x0']<miss_line_new[1] and element['top']>start_top and element['top']<max_top:
        miss_line.append(element['text'])

In [25]:
# 寻找遗失的行
miss_line = [87591,1323000,43905]
miss_line_new = []
for i in miss_line:
    for ele in tlist:
        if str(i) == ele['text']:
            miss_line_new.append(ele)
miss_line = []
for ele in miss_line_new:
    ele['num'] = []
    for element in tlist:
        if element['x0']>first_range[1] and spacing(element['doctop'],ele['doctop'])<5 and re.search('^\d+$',ele['text']):
            miss_line.append(element['text'])

In [160]:
# 按ctrl填空
iter_line = iter(miss_line)
while True:
    if keyboard.is_pressed('ctrl'):
        app.selection.value =next(iter_line)
        time.sleep(0.3)
    if keyboard.is_pressed('win'):
        break

In [32]:
# 删除区域非数字
data = sheet.range('Q2:U54').value
for row in data:
    for i in range(len(row)):
        if isinstance(row[i], str):
            row[i] = re.sub(r'[il]', '1', row[i])
            row[i] = re.sub(r'[^0-9\.]+', '', row[i])

# 将处理后的数据写入 Excel 文件
sheet.range('V2').value = data